In [ ]:
import matplotlib.pyplot as plt

import tensorflow as tf

to_categorical = tf.keras.utils.to_categorical
load_model = tf.keras.models.load_model

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff

import numpy as np

In [ ]:
x, y = make_blobs(n_samples=3000, centers=3, n_features=2, cluster_std=2, random_state=2)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.4, shuffle= True)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
x_train_pd = pd.DataFrame(x_train, columns=['x', 'y'])
y_train_pd = pd.DataFrame(y_train, columns=['class'])

df = pd.concat([x_train_pd, y_train_pd], axis=1)
df["class"] = df["class"].astype(str)

In [ ]:
fig = px.scatter(df, x="x", y="y", color="class")
fig.show()

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(50, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
his = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=1)

In [ ]:
h1 = go.Scatter(y=his.history['loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="acc"
                   )
h2 = go.Scatter(y=his.history['val_loss'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_acc"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))

fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
h1 = go.Scatter(y=his.history['accuracy'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="acc"
                   )
h2 = go.Scatter(y=his.history['val_accuracy'], 
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_acc"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))

fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
predicted_classes = np.argmax(model.predict(x_val), axis=-1)
predicted_classes.shape

In [ ]:
labels = ['A', 'B', 'C']

In [ ]:
y_true = np.argmax(y_val, axis=1)

In [ ]:
cm = confusion_matrix(y_true, predicted_classes)

In [ ]:
def cm_plot(cm, labels):
    x = labels
    y = labels

    z_text = [[str(y) for y in x] for x in cm]
    fig = ff.create_annotated_heatmap(cm, x=x, y=y, annotation_text=z_text, colorscale='blues')

    fig.update_layout(title_text='Confusion Matrix')

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted Value",
                            xref="paper",
                            yref="paper"
                           ))

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=-0.20,
                            y=0.5,
                            showarrow=False,
                            text="Real Value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"
                           ))

    fig.update_layout(margin=dict(t=50, l=200))
    fig['layout']['yaxis']['autorange'] = "reversed"

    fig['data'][0]['showscale'] = True
    fig.show()

In [ ]:
cm_plot(cm, labels)

In [ ]:
print(classification_report(y_true, predicted_classes, target_names=labels, digits=4))

In [ ]:
filepath='model1.h5'
model.save(filepath)

In [ ]:
predict_model = load_model(filepath) 
predict_model.summary()

In [ ]:
a = np.array([[-0.210738, -13.1719]])

predicted_score = predict_model.predict(a)

predicted_score

In [ ]:
predicted_classes = np.argmax(predicted_score, axis=-1)

predicted_classes

In [ ]:
# Copy Model ที่ Train แล้วไปยัง Folder basic_model/model_deploy/python (บน Windows)

# copy model1.h5 .\..\model_deploy\python

# Testing the API Request

In [ ]:
import requests

from requests.auth import HTTPBasicAuth

In [ ]:
URL = 'http://localhost:7001/getclass'

data = {
  'x': -0.210738, 
  'y': -13.1719
}

response = requests.post(URL, json=data, auth=HTTPBasicAuth('nuttachot', 'password'))

In [ ]:
if response.status_code == 200:
    res = response.json()['results']
    print(res)